- Layer: Tensors containing weights and biases
- Block: A collection of Layer(s)
- Model: A collection of Block(s)

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [2]:
X = tf.random.uniform((2, 20))
X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.92525697, 0.773211  , 0.68588674, 0.8745736 , 0.10432208,
        0.91466355, 0.7986289 , 0.13201737, 0.02452648, 0.8426039 ,
        0.22167945, 0.55440986, 0.1372242 , 0.14378846, 0.16490567,
        0.9838227 , 0.9022243 , 0.5898287 , 0.7192172 , 0.88387775],
       [0.5680934 , 0.4133891 , 0.9797405 , 0.08352768, 0.7439598 ,
        0.09695315, 0.46045196, 0.11904716, 0.45413375, 0.7237234 ,
        0.91090333, 0.12395406, 0.09862816, 0.25762403, 0.9489399 ,
        0.37513828, 0.51100135, 0.50811934, 0.2830757 , 0.8499315 ]],
      dtype=float32)>

- STEP 1. $\text{Input}(2, 20)$ matmul $\text{W1}(20, 256)$ add $\text{B1}(256, )$ = $\text{H1}(2, 256)$
- STEP 2. $\text{Input}(2, 256)$ matmul $\text{W1}(256, 10)$ add $\text{B1}(10, )$ = $\text{Output}(2, 10)$

In [3]:
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 3.32431607e-02, -4.13149595e-04,  2.03950584e-01,
        -1.14014581e-01,  1.01974025e-01,  2.52847411e-02,
         2.08812758e-01, -3.57229203e-01,  4.77440566e-01,
        -2.30183586e-01],
       [ 5.57306409e-03, -7.38969743e-02, -2.48782262e-02,
         7.96007738e-03,  1.76981837e-01, -7.24492222e-02,
         8.92012119e-02, -3.21096063e-01,  4.81947869e-01,
        -9.77884084e-02]], dtype=float32)>

### A custom Block

In [4]:
class MLP(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    def call(self, X):
        return self.out(self.hidden((X)))

In [5]:
X = tf.random.uniform((2, 20))
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.16142166,  0.25166762, -0.2747844 , -0.2710586 , -0.13810268,
         0.099378  , -0.01249335, -0.01496436, -0.02269685, -0.3190307 ],
       [-0.33104146,  0.39987943, -0.39303005, -0.09266109,  0.21046634,
        -0.34662175,  0.12212081, -0.29805115,  0.06423774, -0.584865  ]],
      dtype=float32)>

### A custom Sequential Block

In [6]:
class MySequential(tf.keras.Model):
    
    def __init__(self, layers):
        super().__init__()
        self.modules = []
        for layer in layers:
            self.modules.append(layer)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [7]:
net = MySequential([
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)
])

In [8]:
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.2211732 , -0.1746389 ,  0.29796305,  0.11522204, -0.338252  ,
         0.02243249,  0.17052418, -0.02390583, -0.14210296, -0.12029371],
       [ 0.14848441, -0.23827437,  0.20153232,  0.18890366, -0.3277736 ,
        -0.11648276,  0.30210274,  0.0184963 , -0.18080087, -0.2551373 ]],
      dtype=float32)>

### Executing Code in the Forward Propagation Function

In [9]:
class MyMLP(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    def call(self, inputs):
        X = self.flatten(inputs)
        Z = self.hidden(X)
        output = self.out(Z)
        
        return tf.nn.relu(output)

In [10]:
X = tf.random.uniform((2, 20))
net = MyMLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.19631869, 0.12059395, 0.        , 0.07199124],
       [0.26711318, 0.        , 0.        , 0.        , 0.        ,
        0.02636323, 0.        , 0.23199594, 0.        , 0.1545168 ]],
      dtype=float32)>

### Assembling blocks

In [11]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

In [12]:
net = tf.keras.Sequential()
net.add(MyMLP())
net.add(tf.keras.layers.Dense(20))
net.add(NestMLP())
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 16), dtype=float32, numpy=
array([[0.0726741 , 0.        , 0.15203883, 0.05367958, 0.1500624 ,
        0.        , 0.        , 0.05358691, 0.02396537, 0.06336297,
        0.        , 0.        , 0.16490781, 0.09826787, 0.        ,
        0.04325541],
       [0.04676851, 0.        , 0.09582558, 0.07049982, 0.12523016,
        0.        , 0.00777088, 0.0583481 , 0.00833781, 0.05500299,
        0.        , 0.        , 0.15392889, 0.06516259, 0.        ,
        0.04606818]], dtype=float32)>

###  Get Parameters

In [13]:
net.get_weights()

[array([[-0.13132404, -0.05879695, -0.03710087, ...,  0.07712677,
          0.09809488,  0.09934302],
        [-0.01740508,  0.00229542,  0.0434846 , ...,  0.08713476,
         -0.08232149, -0.13698737],
        [ 0.09046409, -0.08304089, -0.07555413, ...,  0.11464128,
          0.13750726, -0.01866268],
        ...,
        [-0.1423376 , -0.10562973, -0.10758974, ...,  0.08031636,
         -0.03848248, -0.08595436],
        [ 0.0517264 , -0.04185463, -0.07376619, ...,  0.01912856,
          0.07157479,  0.03547961],
        [-0.08816464, -0.0294954 , -0.04277567, ..., -0.01138346,
         -0.10955693,  0.03162207]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [14]:
len(net.get_weights())

12

### Custom Layer

In [15]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [16]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.0237263 ,  0.00974653, -0.05903875],
        [-0.05058428, -0.04621651, -0.09449741],
        [ 0.0130282 ,  0.07100022,  0.00302864],
        [ 0.03604994, -0.01382205, -0.04527801],
        [-0.03700906,  0.01659205,  0.02517885]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [17]:
net = tf.keras.Sequential()
net.add(MyDense(256))
net.add(MyDense(10))
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.08518481, 0.08982752, 0.06631047, 0.        , 0.        ,
        0.02913349, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.01439599, 0.05853878, 0.02047709, 0.        ,
        0.0591464 , 0.        , 0.        , 0.        , 0.00932322]],
      dtype=float32)>

In [18]:
net = MySequential([
    MyDense(256),
    MyDense(10)
])
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.        , 0.01890383, 0.01339493, 0.05463038, 0.04197441,
        0.07735933, 0.        , 0.        , 0.01717663, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.00579105,
        0.1104293 , 0.01810639, 0.00994837, 0.08951774, 0.        ]],
      dtype=float32)>